##### Used Source: https://www.laurentluce.com/posts/twitter-sentiment-analysis-using-python-and-nltk/

In [ ]:
import nltk
import re
# http://tweepy.readthedocs.io/en/v3.5.0/index.html
import tweepy
from tweepy import OAuthHandler
# https://pandasguide.readthedocs.io/en/latest/
import pandas as pd
# https://numpy.readthedocs.io/en/latest/
import numpy as np
# https://api.mongodb.com/python/current/
import pymongo

from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import csv

#nltk data for tokenizing and stoppword removal
nltk.download('stopwords')
nltk.download('subjectivity')
nltk.download('punkt')

## Prepare Train Data

In [ ]:
train_data = []

with open('./files/train_data.csv','rt') as csv_data:
    reader = csv.reader(csv_data, delimiter=';')
    for labeled_tweet in reader:
        if labeled_tweet[1] == '0':
            labeled_tweet[1] = 'negative'
        else:
            labeled_tweet[1] = 'positive'
        train_data.append(labeled_tweet)
        

In [ ]:
# Clean tweet and remove links and other unwanted information
def clean_tweet(tweet):
    return ''.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet))

In [ ]:
train_data_cln = [[clean_tweet(labeled_tweet[0]),labeled_tweet[1]] for labeled_tweet in train_data]
tokenized_train_data = [(nltk.tokenize.word_tokenize(labeled_tweet[0]),labeled_tweet[1].lower()) for labeled_tweet in train_data_cln]

print(tokenized_tweets[0])

In [ ]:
stopwords = set(nltk.corpus.stopwords.words('english'))
for (tweet,_) in tokenized_train_data:
    for word in tweet:
        if word in stopwords:
            tweet.remove(word)
    
print(tokenized_train_data[0])

In [ ]:
def get_words_in_tweets(tweets):
    all_words = []
    for (words, sentiment) in tweets:
        all_words.extend(words)
    return all_words

def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    #word_features = wordlist.most_common()
    word_features = wordlist.keys()
    return word_features


word_features = get_word_features(get_words_in_tweets(tokenized_train_data))

In [ ]:
def extract_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in document_words)
    return features

In [ ]:
training_set = nltk.classify.apply_features(extract_features, tokenized_train_data)
print(training_set[0])

## Train Classifier (~5-10 min.)

In [ ]:
# ETT (Estimated Time of Training) = ~ 5-10 min. => depending on Hardware and Docker settings
classifier = nltk.NaiveBayesClassifier.train(training_set)

## Test Classifier

In [ ]:
tweet_student1 = 'I hate FH JOANNEUM'
tweet_student2 = 'FH JOANNEUM is awesome'

print("Student1's sentiment:", classifier.classify(extract_features(tweet_student1.split())))
print("Student2's sentiment:", classifier.classify(extract_features(tweet_student2.split())))

## Test Classifier with real world Tweets

In [ ]:
# Set the received credentials for your recently created TwitterAPI
CONSUMER_KEY = 'MmiELrtF7fSp3vptCID8jKril'
CONSUMER_SECRET = 'HqtMRk4jpt30uwDOLz30jHqZm6TPN6rj3oHFaL6xFxw2k0GkDC'
ACCESS_TOKEN = '116725830-rkT63AILxR4fpf4kUXd8xJoOcHTsGkKUOKSMpMJQ'
ACCESS_TOKEN_SECRET = 'eKzxfku4GdYu1wWcMr5iusTmhFT35cDWezMU2Olr5UD4i'

# auth with your provided 
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

# Create an instance for the TwitterApi
twitter = tweepy.API(auth)

In [ ]:
SCREEN_NAME = "realDonaldTrump"
COUNT = 500

#extract tweets from a user's timeline
tweets = twitter.user_timeline(screen_name=SCREEN_NAME, count=COUNT)
print("{} tweets extracted.\n".format(len(tweets)))

In [ ]:
# Clean tweet and remove links and other unwanted information
def clean_tweet(tweet):
    return ''.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet))


tweets = [clean_tweet(tweet.text) for tweet in tweets]

In [ ]:
# Create a pandas DataFrame out of the tweets
data = pd.DataFrame(data=[t for t in tweets], columns=['Tweets'])

# Diplay the first 5 elements of the DataFrame
display(data.head(5))

In [ ]:
tokenized_tweets = [nltk.tokenize.word_tokenize(tweet.lower()) for tweet in tweets]
print(tokenized_tweets[0])

In [ ]:
stopwords = set(nltk.corpus.stopwords.words('english'))
for tweet in tokenized_tweets:
    for word in tweet:
        if word in stopwords:
            tweet.remove(word)

print(tokenized_tweets[0])

In [ ]:
trump_tweet = tokenized_tweets[90]
print(tweets[90])
print("Trump's Sentiment about the Tweet:", classifier.classify(extract_features(trump_tweet)).upper())